In [1]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import layers , activations , models , preprocessing , utils
import tensorflow as tf
from  sklearn.model_selection import train_test_split

In [2]:
#doc du lieu ve dang data-label
data=[]
label=[]
error=0
with open('yelp_labelled.csv','r') as f:
    for line in f:
        part = line.strip().strip(',').split(',')
        if len(part)<=1:
            error+=1
            continue
        if part[-1]!='0' and part[-1]!='1':
            error+=1
            continue
        data.append(", ".join(part[:-1]))
        label.append(int(part[-1]))
print('Read',len(data),'items, skiped',error,'error lines')
print('Label 0:',len(label)-sum(label),'Label 1:',sum(label))

Read 992 items, skiped 8 error lines
Label 0: 496 Label 1: 496


In [3]:
print(data[:5])
print(label[:5])


['Wow... Loved this place.', 'Crust is not good.', 'Not tasty and the texture was just nasty.', 'Stopped by during the late May bank holiday off Rick Steve recommendation and loved it.', 'The selection on the menu was great and so were the prices.']
[1, 0, 0, 1, 1]


In [4]:
tokenizer = tf.keras.preprocessing.text.Tokenizer(lower=True,split=' ')
tokenizer.fit_on_texts(data)
X = tokenizer.texts_to_sequences(data)
X = pad_sequences(X)
print(X[0])

[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0 424 164   8  15]


In [5]:
print("input shape",X.shape)
n_vocab=len(tokenizer.word_index)+1
print('Vocab count',n_vocab)

input shape (992, 32)
Vocab count 2054


In [6]:
print(n_vocab)
print(X.shape[1])

2054
32


In [7]:
embedding_dim = 128
maxlen = X.shape[1]
vocab_size = n_vocab 

inputs = layers.Input(shape=( maxlen , ))
embedding = layers.Embedding(vocab_size, embedding_dim, input_length=maxlen)(inputs)

cnn1=layers.Conv1D(128, 1, activation='relu')(embedding)
cnn1 = layers.GlobalMaxPooling1D()(cnn1)

cnn2=layers.Conv1D(128, 2, activation='relu')(embedding)
cnn2 = layers.GlobalMaxPooling1D()(cnn2)

outputs = layers.Concatenate()([cnn1,cnn2])

outputs = layers.Dense(24, activation='tanh')(outputs)
outputs = layers.Dense(12, activation='tanh')(outputs)
outputs = layers.Dense(2, activation='softmax')(outputs)
model=models.Model(inputs,outputs)
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 32)]         0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 32, 128)      262912      input_1[0][0]                    
__________________________________________________________________________________________________
conv1d (Conv1D)                 (None, 32, 128)      16512       embedding[0][0]                  
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 31, 128)      32896       embedding[0][0]                  
______________________________________________________________________________________________

In [8]:
#tách train test
Y = tf.keras.utils.to_categorical(label)
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.25, random_state = 36)

In [9]:
history = model.fit(X_train, y_train,
                    epochs=10,
                    verbose=1,
                    validation_data=(X_test, y_test),
                    batch_size=20)

Epoch 1/10
38/38 [==============================] - 2s 28ms/step - loss: 0.6873 - accuracy: 0.5425 - val_loss: 0.6505 - val_accuracy: 0.6492
Epoch 2/10
38/38 [==============================] - 1s 14ms/step - loss: 0.5603 - accuracy: 0.8507 - val_loss: 0.4910 - val_accuracy: 0.7823
Epoch 3/10
38/38 [==============================] - 1s 14ms/step - loss: 0.2585 - accuracy: 0.9528 - val_loss: 0.4207 - val_accuracy: 0.8024
Epoch 4/10
38/38 [==============================] - 1s 15ms/step - loss: 0.1021 - accuracy: 0.9804 - val_loss: 0.4415 - val_accuracy: 0.8105
Epoch 5/10
38/38 [==============================] - 1s 13ms/step - loss: 0.0470 - accuracy: 0.9985 - val_loss: 0.4635 - val_accuracy: 0.8226
Epoch 6/10
38/38 [==============================] - 1s 14ms/step - loss: 0.0300 - accuracy: 0.9980 - val_loss: 0.4976 - val_accuracy: 0.8145
Epoch 7/10
38/38 [==============================] - 1s 13ms/step - loss: 0.0193 - accuracy: 1.0000 - val_loss: 0.5177 - val_accuracy: 0.8266
Epoch 8/10
38

In [10]:
loss, accuracy = model.evaluate(X_train, y_train, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(X_test, y_test, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))

Training Accuracy: 1.0000
Testing Accuracy:  0.8226
